In [34]:
import torch
import torch.nn as nn
import random
import pandas as pd
import numpy as np
import os
import tqdm.notebook as tq
import torch.nn.functional as F
from torch.backends import cudnn
from torch.utils.data import DataLoader, Dataset
from tqdm.autonotebook import tqdm
import torch.nn.functional as F
import torch.optim as optim
import math
import datetime

# Config

In [35]:
days_ahead = 11

# Load data

## Timeseries

In [36]:
ts60 = pd.read_csv('../VEOLIA/artifacts/timeseries_60min.csv', index_col=0, parse_dates=True)[datetime.datetime(2020, 11, 6):]
ts60.head(3)

,Diff Load Activa Total (60 minuto),Diff Load Primario ACS (60 minuto),Diff Load Consumo ACS (60 minuto),Diff Load Caldera 1 (60 minuto),Diff Load Caldera 2 (60 minuto),True Load Caldera 1 (60 minuto),True Load Caldera 2 (60 minuto),True Load Primario ACS (60 minuto),True Load Consumo ACS (60 minuto),Energía Caldera 1 (60 minuto),...,TEMPERATURA DEPOSITO 2 ACS (60 minuto),DEMANDA CALEFACCIÓN (60 minuto),DEMANDA ACS (60 minuto),DEMANDA QUEMADORES (60 minuto),Caudal Caldera 1 (60 minuto),Caudal Caldera 2 (60 minuto),Caudal Primario ACS (60 minuto),Caudal Consumo ACS (60 minuto),CONTADOR GAS CALDERA 1 (60 minuto),CONTADOR GAS CALDERA 2 (60 minuto)
datetime,,,,,,,,,,,,,,,,,,,,,
2020-11-06 00:00:00,2600.0,10.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,38.38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6705.2,12467.6
2020-11-06 01:00:00,2610.0,10.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,38.39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6723.8,12467.6
2020-11-06 02:00:00,2570.0,20.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,38.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6736.1,12467.6


### Choose timeseries variable

In [37]:
tal = pd.DataFrame(ts60['Diff Load Activa Total (60 minuto)'], index=ts60.index)
tal.head()

,Diff Load Activa Total (60 minuto)
datetime,
2020-11-06 00:00:00,2600.0
2020-11-06 01:00:00,2610.0
2020-11-06 02:00:00,2570.0
2020-11-06 03:00:00,2580.0
2020-11-06 04:00:00,2600.0


## Time features

In [38]:
calendar60 = pd.read_csv('../VEOLIA/artifacts/time_60min.csv', index_col=1, parse_dates=True)[datetime.datetime(2020, 11, 6):].drop(columns='Unnamed: 0')
calendar60.head(3)

,year,month,day,hour,minute,second,weekday,weekend,holiday
datetime,,,,,,,,,
2020-11-06 00:00:00,2020,11,6,0,0,0,4,False,False
2020-11-06 01:00:00,2020,11,6,1,0,0,4,False,False
2020-11-06 02:00:00,2020,11,6,2,0,0,4,False,False


### Choose subset of time variables
Minutes and seconds are dropped as we have an hourly dataset

In [44]:
time = calendar60.drop(columns=['minute', 'second'])
time.head()

,year,month,day,hour,weekday,weekend,holiday
datetime,,,,,,,
2020-11-06 00:00:00,2020,11,6,0,4,False,False
2020-11-06 01:00:00,2020,11,6,1,4,False,False
2020-11-06 02:00:00,2020,11,6,2,4,False,False
2020-11-06 03:00:00,2020,11,6,3,4,False,False
2020-11-06 04:00:00,2020,11,6,4,4,False,False


## Weather features

In [43]:
weather = pd.read_csv('../VEOLIA/artifacts/weather_curated_60min.csv', index_col=0, parse_dates=True)[datetime.datetime(2020, 11, 1):]
weather.head(3)

,temp,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_1h,clouds_all,weather_id
datetime,,,,,,,,,,,,,
2020-11-01 00:00:00,9.42,9.42,7.63,10.36,1028,99,1.0,140,0.0,0.0,0.0,0,800
2020-11-01 01:00:00,9.96,8.41,7.73,11.47,1028,96,3.1,180,0.0,0.0,0.0,0,800
2020-11-01 02:00:00,10.31,9.93,8.03,11.47,1027,97,5.7,190,0.0,0.0,0.0,0,800


## Create final dataframe 

In [48]:
df = tal.merge(time, left_index=True, right_index=True).merge(weather, left_index=True, right_index=True)
df.head()

,Diff Load Activa Total (60 minuto),year,month,day,hour,weekday,weekend,holiday,temp,feels_like,...,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_1h,clouds_all,weather_id
datetime,,,,,,,,,,,,,,,,,,,,,
2020-11-06 00:00:00,2600.0,2020,11,6,0,4,False,False,9.13,6.24,...,11.13,1021,92,5.7,70,0.0,0.0,0.0,97,500
2020-11-06 01:00:00,2610.0,2020,11,6,1,4,False,False,10.97,10.19,...,11.58,1020,79,7.2,130,0.0,0.0,0.0,75,803
2020-11-06 02:00:00,2570.0,2020,11,6,2,4,False,False,11.09,10.34,...,11.48,1020,80,7.7,140,0.0,0.2,0.0,0,500
2020-11-06 03:00:00,2580.0,2020,11,6,3,4,False,False,10.67,9.93,...,11.03,1020,82,6.2,140,0.0,0.0,0.0,0,800
2020-11-06 04:00:00,2600.0,2020,11,6,4,4,False,False,11.39,10.65,...,11.94,1019,79,6.2,130,0.0,0.0,0.0,0,800


In [42]:
my_seed = 1
random.seed(my_seed)
np.random.seed(my_seed)
torch.manual_seed(my_seed)
cudnn.benchmark = True
torch.backends.cudnn.deterministic = True

gpu_device = 0
if torch.cuda.is_available():
    device = 'cuda:0'
    print(torch.cuda.get_device_name(0))
    torch.cuda.manual_seed(my_seed)
    print(torch.cuda.get_device_properties(0))
else:
    device = 'cpu'

GeForce RTX 2060
_CudaDeviceProperties(name='GeForce RTX 2060', major=7, minor=5, total_memory=6144MB, multi_processor_count=30)


In [280]:
df.head(10)

,Unnamed: 0,SHIP_ID,LON,LAT,HEADING,TIMESTAMP,LENGTH,WIDTH
0,0,63460,5.290505,60.395870,511,1.294757e+12,26,9
1,1,63460,5.312657,60.392750,511,1.294758e+12,26,9
2,2,63460,5.265542,60.399731,511,1.294759e+12,26,9
3,3,63460,5.313068,60.392410,511,1.294759e+12,26,9
4,4,63460,5.272945,60.399330,511,1.294760e+12,26,9
5,5,63460,5.313075,60.392429,511,1.294761e+12,26,9
6,6,63460,5.273248,60.398289,511,1.294761e+12,26,9
7,7,63460,5.227423,60.406509,511,1.294764e+12,26,9
8,8,63460,5.227420,60.406490,511,1.294765e+12,26,9
9,9,63460,5.227388,60.406490,511,1.294766e+12,26,9


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5688 entries, 2020-11-06 00:00:00 to 2021-06-30 23:00:00
Data columns (total 21 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Diff Load Activa Total (60 minuto)  5688 non-null   float64
 1   year                                5688 non-null   int64  
 2   month                               5688 non-null   int64  
 3   day                                 5688 non-null   int64  
 4   hour                                5688 non-null   int64  
 5   weekday                             5688 non-null   int64  
 6   weekend                             5688 non-null   bool   
 7   holiday                             5688 non-null   bool   
 8   temp                                5688 non-null   float64
 9   feels_like                          5688 non-null   float64
 10  temp_min                            5688 non-null   float64
 11  temp_ma

# Converting to PyTorch Dataset

In [78]:
batch_size = 2
entry_size = 13
hidden_size = 128
epochs = 10
learning_rate = 0.003
pi_tensor = torch.Tensor([math.pi]).to(device)

In [79]:
class ShipsDataset(Dataset):
    def __init__(self, df):
        # self.trajectories = []
        # for group in tqdm(list(df.groupby(['SHIP_ID']))[:10000]):
        #     trajectory = []
        #     preds = []
        #     # print(type(group[1]))
        #     data = group[1][['LON', 'LAT', 'TIMESTAMP', 'HEADING']].to_numpy()
        #     # print(data)
        #     for i in range(1, len(data) - 2):
        #         tmp = np.append(data[i-1], data[i])
        #         entry = np.append(tmp, data[i+1])
        #         # print(entry, data[i+2])
        #         trajectory.append([torch.from_numpy(entry), torch.from_numpy(data[i+2])])
        #     self.trajectories.append(trajectory)


        self.data = []
        tensor_data = torch.from_numpy(df.to_numpy(dtype='float32'))
        print(tensor_data)
        print('=============')
        for i in tqdm(range(1, int(len(tensor_data)) - 2)):
            entry = torch.cat((tensor_data[i-1], tensor_data[i], tensor_data[i+1], tensor_data[i+2][3].view(-1)), 0)
            self.data.append([entry, tensor_data[i+2][:3].view(-1)])


    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        print(self.data[idx])
        return self.data[idx]


In [85]:
trajectories = ShipsDataset(df)
loader = DataLoader(trajectories, batch_size=batch_size, shuffle=True)
print(next(iter(loader)))

tensor([[2600., 2020.,   11.,  ...,    0.,   97.,  500.],
        [2610., 2020.,   11.,  ...,    0.,   75.,  803.],
        [2570., 2020.,   11.,  ...,    0.,    0.,  500.],
        ...,
        [1930., 2021.,    6.,  ...,    0.,    0.,  800.],
        [2350., 2021.,    6.,  ...,    0.,    0.,  800.],
        [2470., 2021.,    6.,  ...,    0.,    0.,  800.]])


100%|██████████| 5685/5685 [00:00<00:00, 47378.34it/s]

[tensor([ 3.5200e+03,  2.0210e+03,  1.0000e+00,  9.0000e+00,  3.0000e+00,
         5.0000e+00,  0.0000e+00,  0.0000e+00, -3.0300e+00, -8.7700e+00,
        -3.6700e+00, -2.9800e+00,  1.0070e+03,  8.6000e+01,  5.1000e+00,
         4.0000e+01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  7.5000e+01,
         6.0000e+02,  3.2800e+03,  2.0210e+03,  1.0000e+00,  9.0000e+00,
         4.0000e+00,  5.0000e+00,  0.0000e+00,  0.0000e+00, -3.0300e+00,
        -9.1300e+00, -4.1700e+00, -2.9800e+00,  1.0070e+03,  8.6000e+01,
         5.7000e+00,  5.0000e+01,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         9.0000e+01,  6.0000e+02,  3.2300e+03,  2.0210e+03,  1.0000e+00,
         9.0000e+00,  5.0000e+00,  5.0000e+00,  0.0000e+00,  0.0000e+00,
        -3.8900e+00, -1.0760e+01, -4.1700e+00, -2.9800e+00,  1.0070e+03,
         9.2000e+01,  6.7000e+00,  4.0000e+01,  0.0000e+00,  0.0000e+00,
         1.0000e+00,  7.5000e+01,  6.0100e+02,  9.0000e+00]), tensor([3.3600e+03, 2.0210e+03, 1.0000e+00])]
[tensor([1.6200

In [81]:
loader = DataLoader(trajectories, batch_size=batch_size, shuffle=True)

# BiDirectional LSTM Model 

In [291]:
class Model(torch.nn.Module):
    def __init__(self, inputs_size, hidden_size, num_layers):
        super(Model, self).__init__()

        self.num_layers = num_layers
        self.hidden_size = hidden_size
        # self.rnn = nn.RNN(inputs_size, hidden_size, num_layers=3, bidirectional=True, batch_first=True)
        self.lstm = nn.LSTM(inputs_size, hidden_size, num_layers=5, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, 3)

    def forward(self, x):
        x = x.float().to(device)
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x) # ----> to check what to pass as the second argument for (c_0, h_0)
        return self.fc(out[:, -1, :])


In [292]:
model = Model(entry_size, hidden_size, 3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [293]:
def myloss(output, target):
    heading_loss = torch.abs(output[:,2] - target[:,2])
    coords_loss = torch.sqrt(((output[:,0:2]-target[:,0:2])**2).sum(axis=1))
    dom = torch.sin(pi_tensor - heading_loss/2)
    return (heading_loss * coords_loss) / dom

In [294]:
batch_norm = nn.BatchNorm1d(1)
labels_norm = nn.BatchNorm1d(3)

In [295]:
losses = []
for epoch in range(epochs):
    # counter = 0
    total_loss = 0
    print('\n\nEpoch:', epoch+1, 'out of', epochs, flush=True)
    progress = tqdm(enumerate(loader), desc="Progress: ", total=len(loader))
    for i, (ais_data, labels) in progress:
        if i == len(loader) - 1:
            pass
        else:
            # print(ais_data, flush=True)
            inputs = ais_data.view(batch_size, 1, entry_size)
            # print(inputs)
            # print(inputs.size())
            # counter += 1
            # print(labels, labels[:,2])
            out  = model(batch_norm(inputs.float()))
            # print(out, flush=True)
            loss = myloss(out, labels_norm(labels.float()).to(device))
            optimizer.zero_grad()
            loss.sum().backward()
            optimizer.step()
            current_loss = loss.sum().item()
            total_loss += current_loss
            progress.set_description("Loss: {:.4f}".format(total_loss/(i+1)))

    losses.append(total_loss / len(loader))
            
        # break



Epoch: 1 out of 10


Epoch: 2 out of 10


Epoch: 3 out of 10


Epoch: 4 out of 10


Epoch: 5 out of 10


Epoch: 6 out of 10


Epoch: 7 out of 10


Epoch: 8 out of 10


Epoch: 9 out of 10


Epoch: 10 out of 10


Progress:   0%|          | 0/8 [00:00<?, ?it/s]

Progress:   0%|          | 0/8 [00:00<?, ?it/s]

Progress:   0%|          | 0/8 [00:00<?, ?it/s]

Progress:   0%|          | 0/8 [00:00<?, ?it/s]

...

Progress:   0%|          | 0/8 [00:00<?, ?it/s]

Progress:   0%|          | 0/8 [00:00<?, ?it/s]

Progress:   0%|          | 0/8 [00:00<?, ?it/s]

Progress:   0%|          | 0/8 [00:00<?, ?it/s]

Progress:   0%|          | 0/8 [00:00<?, ?it/s]

In [296]:
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': total_loss,
            }, os.path.join('models/', 'epoch-{}.pth'.format(epoch)))
torch.save(model, os.path.join('models', f'epoch-{epoch}.pt'))

In [297]:
# model = torch.load('to montelo moy')
# inputs = data tou xrhsth
#
# outputs = model(inputs)
# outputs = [pred_lat, pred_lon, pred_heading]